In [0]:

import pyspark
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructType,StructField, StringType
from pyspark.sql.functions import sum,avg,max,count,mean
import pyspark.sql.functions as f


In [0]:
spark = SparkSession.builder.appName('groupy_by').getOrCreate()

<h3>Criação de dataframes</h3> <br>
Abaixo uma simples massa de dados foi criada para utilização nos exercicios.</br>
O dataframe em questão consiste em dados de supostos games, informando <h6>nome, empresa, tipo, data de produção, sistema, custo de produção, receita e classificação</h6>



In [0]:
columns = ['title','company','type','production_year','system','production_cost','revenue','rating']

data = [
('Blasting Boxes','Simone Games','action adventure',1998,'PC',100000,200000,7),
('Run Run Run!','13 Mad Bits','shooter',2011,'PS3',3500000,	650000,	3),
('Duck n’Go','13 Mad Bits','shooter',2012,'Xbox',3000000,1500000,5),
('SQL Wars!','Vertabelo','wargames',2017,'Xbox',5000000,25000000,10),
('Tap Tap Hex!','PixelGaming Inc.','rhythm',2006,'PS2',2500000,3500000,7),
('NoRisk','Simone Games','action adventure',2004,'PS2',1400000,3400000,8)
  ]

df_games = spark.createDataFrame(data,columns)


In [0]:
df_games.display()

title,company,type,production_year,system,production_cost,revenue,rating
Blasting Boxes,Simone Games,action adventure,1998,PC,100000,200000,7
Run Run Run!,13 Mad Bits,shooter,2011,PS3,3500000,650000,3
Duck n’Go,13 Mad Bits,shooter,2012,Xbox,3000000,1500000,5
SQL Wars!,Vertabelo,wargames,2017,Xbox,5000000,25000000,10
Tap Tap Hex!,PixelGaming Inc.,rhythm,2006,PS2,2500000,3500000,7
NoRisk,Simone Games,action adventure,2004,PS2,1400000,3400000,8


In [0]:

df_games.groupBy('company').count().display()


company,count
Simone Games,2
13 Mad Bits,2
Vertabelo,1
PixelGaming Inc.,1


In [0]:

df_games.groupBy('company').agg(
    count('*').alias('qtd')
).display()

company,qtd
Simone Games,2
13 Mad Bits,2
Vertabelo,1
PixelGaming Inc.,1


<h4>Exercício 1: Receita total de cada empresa</h4>
Exercício:

Obtenha o nome e a receita total de cada empresa.

Solução:

SELECT
  company,
  SUM(revenue)
FROM games
GROUP BY company;

company,soma_receita
Simone Games,3600000
13 Mad Bits,2150000
Vertabelo,25000000
PixelGaming Inc.,3500000


<h4>Exercício 2: Jogos produzidos por ano com receita e custo médios</h4>
Exercício:

Gere um relatório com o ano de produção e o número de jogos lançados nesse ano (denominado count), a média do custo de produção de todos os jogos produzidos nesse ano (denominado avg_cost) e a receita média desse ano (denominado avg_revenue).
Solução:

SELECT  
  production_year,
  COUNT(*) AS count,
  AVG(production_cost) AS avg_cost,
  AVG(revenue) AS avg_revenue
FROM games
GROUP BY production_year;

production_year,count,avg_cost,avg_revenue
1998,1,100000.0,200000.0
2011,1,3500000.0,650000.0
2012,1,3000000.0,1500000.0
2017,1,5000000.0,2.5E7
2006,1,2500000.0,3500000.0
2004,1,1400000.0,3400000.0


type,count,avg_cost,avg_revenue
action adventure,2,750000.0,1800000.0
shooter,2,3250000.0,1075000.0
wargames,1,5000000.0,2.5E7
rhythm,1,2500000.0,3500000.0


<h3>Exercício 3: Número de jogos lucrativos de cada tipo de jogo</h3>
Exercício:

Conte quantos jogos de um determinado tipo são lucrativos (ou seja, a receita foi maior que o custo de produção). Mostre o tipo de jogo e o número de jogos lucrativos (denominado number_of_games) para cada tipo.

Solução:

SELECT   
  type,
  COUNT(*) AS number_of_games
FROM games
WHERE revenue > production_cost
GROUP BY type;

type,count
action adventure,2
wargames,1
rhythm,1


<h4>Exercício 4: Receita total por tipo de jogo nos sistemas PS2 e PS3</h4>
Exercício:

Obtenha o tipo de jogo e a receita total gerada para jogos com um production_year após 2010 e com um PS2 ou PS3 system. Ordene o resultado de modo que os tipos com a maior receita venham primeiro.

Solução:

SELECT
  type,
  SUM(revenue) AS total_revenue
FROM games
WHERE production_year > 2010 AND systems IN ('PS2','PS3')
GROUP BY type;

type,total_revenue
shooter,650000
rhythm,3500000


Exercício 5: Lucro bruto total por empresa
Exercício:

Para todas as empresas presentes na tabela, obtenha seus nomes e a soma do lucro bruto em todos os anos. (Suponha que o lucro bruto = receita - custo de produção). Nomeie essa coluna como gross_profit_sum. Ordene os resultados por lucro bruto, em ordem decrescente.

Solução:

SELECT
  company,
  SUM(revenue - production_cost) AS gross_profit_sum
FROM games
GROUP BY company
ORDER BY gross_profit_sum DESC;

company,gross_profit
13 Mad Bits,4350000
PixelGaming Inc.,-1000000
Simone Games,-2100000
Vertabelo,-20000000


<h4>Exercício 6: Lucro bruto anual por empresa</h4>
Exercício:

Obter o lucro bruto anual de cada empresa. Em outras palavras, queremos um relatório com o nome da empresa, o ano e o lucro bruto desse ano. Ordene o relatório por nome da empresa e ano.

Solução:

SELECT
  company,
  production_year,
  SUM(revenue - production_cost) AS gross_profit_sum
FROM  games
GROUP BY company, production_year
ORDER BY company, production_year;

company,production_year,gross_profit
13 Mad Bits,2011,2850000
13 Mad Bits,2012,1500000
PixelGaming Inc.,2006,-1000000
Simone Games,1998,-100000
Simone Games,2004,-2000000
Vertabelo,2017,-20000000


<h4>Exercício 8: Empresas que produzem mais de um jogo</h4>
Exercício:

Para cada empresa, selecione seu nome, o número de jogos que produziu (como a coluna number_of_games ) e o custo médio de produção (como a coluna avg_cost ). Mostre apenas as empresas que produzem mais de um jogo.

Solução:

SELECT
  company,
  COUNT(company) AS number_of_games,
  AVG(production_cost) AS avg_cost
FROM games
GROUP BY company
HAVING COUNT(*) > 1;

company,number_of_games,avg_cost
Simone Games,2,750000.0
13 Mad Bits,2,3250000.0


<h4>Exercício 9: Empresas que produzem jogos "bons" com receita superior a 4 milhões</h4>
Exercício:

Estamos interessados em bons jogos produzidos entre 2000 e 2009. Um bom jogo é um jogo lucrativo com uma classificação superior a 6. Para cada empresa, mostre o nome da empresa, sua receita total de jogos bons produzidos entre 2000 e 2009 (como a coluna revenue_sum ) e o número de jogos bons que ela produziu nesse período (como a coluna number_of_games ). Mostre apenas as empresas com receita de jogos de qualidade superior a 4.000.000.

Solução:

SELECT
  company,
  COUNT(company) AS number_of_games,
  SUM(revenue) AS revenue_sum
FROM games
WHERE production_year BETWEEN 2000 AND 2009
  AND rating > 6
  AND revenue - production_cost > 0
GROUP BY company
HAVING SUM(revenue) > 4000000;

company,number_of_games,revenue_sum
PixelGaming Inc.,1,3500000
Simone Games,1,3400000
